In [126]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import shutil
import tensorflow as tf
import pathlib
import PIL
import time
import zipfile
import random
from tensorflow import keras as ks
from tensorflow.keras.layers import *
from keras.utils.np_utils import to_categorical 
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import warnings
from tqdm.notebook import tqdm
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
import regex as re
import time
import gc
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
warnings.filterwarnings('ignore')

In [127]:
sub=pd.read_csv("/content/submission_sample.csv", sep=";", engine="python")
train_df=pd.read_csv("/content/Xy_train.csv", sep=";",engine="python")
test_df=pd.read_csv("/content/X_test.csv", sep=";",engine="python")

In [129]:
train_df.head()

,_id,categ_0,txt_0,categ_1,num_0,categ_2,categ_4,categ_5,num_5,categ_8,...,categ_6,categ_7,txt_3,num_4,categ_3,txt_1,num_6,num_2,num_7,y
0,625267cf0a63071b21c47e2b,2,"a:o@`''E]@W@`'']@eBcqK~@mo""c<@1!)@`RRK@Kw",4,168000,1,0,49,NaN,4,...,5,2,<o<,2008.0,2368,<o<,NaN,NaN,NaN,4500.0
1,625666e07a2d1ced83779994,2,"a:o@eKc<:`@Z!)@K@""]:@R:e:c<@Oq'Yeoee@+:`DKoee@...",4,50,3,0,50,115.0,41,...,5,2,"gEO+3'KccKDEb@Eq'""3+oK'3q@]'@R:K'ee'Eb@Eqo]oq@...",2022.0,2368,<o<,NaN,998.0,NaN,22133.0
2,61b9ed1f7e264de96093187b,1,"a:o@eKc<:`@+o3<`D@']:K:c<@Z!)@Kd""]:@Z1)",4,85717,3,0,34,115.0,41,...,1,2,g}`oY'qo@]Eo:]'@o3@eKoK:c<<'Y'<K}b@}`oBK'3qe@]...,2017.0,2296,"R'D:`3+'@`c<Kqc+'@d@""oqo<K:'@;@Yc:e@:<`+3e'@G'...",NaN,NaN,6.0,11490.0
3,624ac4947e9fdee78b7eba63,2,"a:o@q:c@Z!)@K@""]:@Z1)@B+oK:<3Y@']:K:c<",4,30000,3,0,49,NaN,33,...,10,1,<o<,2017.0,2368,<o<,NaN,NaN,NaN,14500.0
4,6254d841bef56d2c1cf934ea,2,"a:o@eBcqKo""'@hoo@e'q:'",2,173000,3,0,49,NaN,39,...,8,1,<o<,2013.0,2368,<o<,NaN,NaN,NaN,10000.0


In [130]:
test_df.head()

,_id,categ_0,txt_0,categ_1,num_0,categ_2,categ_4,categ_5,num_5,categ_8,...,num_10,categ_6,categ_7,txt_3,num_4,categ_3,txt_1,num_6,num_2,num_7
0,6251210db22e614680cef17b,2,a:o@`oq'<e@@@1!)@`RRK@w@Kqo@o:q`c,4,168982,3,0,49,NaN,1,...,20.0,11,2,<o<,2007.0,2368,<o<,NaN,NaN,NaN
1,609a41b746383f48ea7ed9c4,2,"a:o@eBcqKo""'@Z!?@`q]:@ZZu@:e""@,w1@o`K:R'",2,74500,3,0,19,119.0,39,...,999.0,1,1,gEo:q@`c<]:K:c<<'Eb@E`oY'qo@]'@q'`3+Eb@EO+3'Kc...,NaN,2368,<o<,NaN,NaN,NaN
2,609b46c346383f48ea85a360,2,"a:o@q:c@Bq'Y:3Y@Z!,+@Z)_",4,82996,3,0,34,114.0,33,...,999.0,1,1,<o<,NaN,2368,<o<,NaN,NaN,NaN
3,609a42ee46383f48ea7ef087,2,a:o@B:`o<Kc@Z!)@;;`D@o`K:R'@uB@`+:Y@Yo<3@3eO@O...,4,36470,3,0,19,105.0,25,...,999.0,1,1,gEO+3'KccKDEb@Ecq]:<oK'3q@]'@Ocq]Es,2016.0,2368,<o<,NaN,NaN,NaN
4,609a42cd46383f48ea7eed93,2,"a:o@eBcqKo""'@1!)@`q]:@Z,)@,w1@.oB@3qOo<@q:]'q",2,185000,3,0,51,184.0,39,...,999.0,1,1,gEo:q@`c<]:K:c<<'Es,NaN,2368,<o<,NaN,NaN,NaN


In [131]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [132]:
test_df.shape

(15000, 27)

In [133]:
Id=test_df["_id"]

In [134]:
train_df.shape

(45000, 28)

In [135]:
train_df["y"].nunique()

5457

In [136]:
train_df.isna().sum().sort_values(ascending =False).head(20)

num_6      43318
num_7      42688
num_9      41739
num_8      35759
num_2      35674
num_5      32416
num_3      30466
num_1      24203
num_4       5276
num_10      1536
y              1
txt_1          0
categ_3        0
txt_3          0
categ_7        0
categ_6        0
_id            0
categ_9        0
date_0         0
categ_0        0
dtype: int64

In [137]:
train_df.isna().sum().sort_values(ascending =False).head(20)/len(train_df)

num_6      0.962622
num_7      0.948622
num_9      0.927533
num_8      0.794644
num_2      0.792756
num_5      0.720356
num_3      0.677022
num_1      0.537844
num_4      0.117244
num_10     0.034133
y          0.000022
txt_1      0.000000
categ_3    0.000000
txt_3      0.000000
categ_7    0.000000
categ_6    0.000000
_id        0.000000
categ_9    0.000000
date_0     0.000000
categ_0    0.000000
dtype: float64

# Features engineering

*I choose to drop text_rows because after encoding them and use other method like tfidf and bag of words , it didn't give me better result*

In [138]:
train_df.drop(["_id","num_6","num_7","num_9","num_8","num_2","num_5","num_3","num_1","txt_0","txt_1","txt_2","txt_3"], axis=1, inplace=True)
test_df.drop(["_id","num_6","num_7","num_9","num_8","num_2","num_5","num_3","num_1","txt_0","txt_1","txt_2","txt_3"], axis=1, inplace=True)


In [139]:
train_df.shape

(45000, 15)

In [140]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45000 entries, 0 to 44999
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   categ_0  45000 non-null  int64  
 1   categ_1  45000 non-null  int64  
 2   num_0    45000 non-null  int64  
 3   categ_2  45000 non-null  int64  
 4   categ_4  45000 non-null  int64  
 5   categ_5  45000 non-null  int64  
 6   categ_8  45000 non-null  int64  
 7   date_0   45000 non-null  object 
 8   categ_9  45000 non-null  int64  
 9   num_10   43464 non-null  float64
 10  categ_6  45000 non-null  int64  
 11  categ_7  45000 non-null  int64  
 12  num_4    39724 non-null  float64
 13  categ_3  45000 non-null  int64  
 14  y        44999 non-null  float64
dtypes: float64(3), int64(11), object(1)
memory usage: 5.1+ MB


In [141]:
train_df = reduce_mem_usage(train_df)
test_df = reduce_mem_usage(test_df)
sub = reduce_mem_usage(sub)
gc.collect()

Memory usage after optimization is: 1.55 MB
Decreased by 70.0%
Memory usage after optimization is: 0.46 MB
Decreased by 71.4%
Memory usage after optimization is: 1.29 MB
Decreased by 6.2%


52

In [142]:
import datetime as dt
train_df["date_0"]=pd.to_datetime(train_df["date_0"])
train_df['Date_year'] =train_df["date_0"].dt.year
train_df['Date_month'] = train_df["date_0"].dt.month
train_df['Date_day'] = train_df["date_0"].dt.day
train_df.drop("date_0", axis=1, inplace=True)
test_df["date_0"]=pd.to_datetime(test_df["date_0"])
test_df['Date_year'] =test_df["date_0"].dt.year
test_df['Date_month'] = test_df["date_0"].dt.month
test_df['Date_day'] = test_df["date_0"].dt.day
test_df.drop("date_0", axis=1, inplace=True)

# Mice

In [143]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
estimator=RandomForestRegressor(max_depth=8)
mice = IterativeImputer(estimator=estimator,random_state=11,skip_complete=True)
imp_train_df=mice.fit_transform(train_df)
imp_train_df=pd.DataFrame(imp_train_df,columns=train_df.columns)

In [144]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
imp_test_df=mice.fit_transform(test_df)
imp_test_df=pd.DataFrame(imp_test_df,columns=test_df.columns)

In [145]:
imp_train_df.isna().sum()

categ_0       0
categ_1       0
num_0         0
categ_2       0
categ_4       0
categ_5       0
categ_8       0
categ_9       0
num_10        0
categ_6       0
categ_7       0
num_4         0
categ_3       0
y             0
Date_year     0
Date_month    0
Date_day      0
dtype: int64

In [146]:
imp_train_df["mean"]=imp_train_df.mean(axis=1)
imp_train_df["count"]=imp_train_df.count(axis=1)
imp_train_df["std"]=imp_train_df.std(axis=1)
imp_train_df["min"]=imp_train_df.min(axis=1)
imp_train_df["max"]=imp_train_df.max(axis=1)
imp_train_df["kurt"]=imp_train_df.kurt(axis=1)
imp_train_df["skew"]=imp_train_df.skew(axis=1)
imp_train_df["q1"]=imp_train_df.quantile(q=0.1,axis=1)
imp_train_df["q2"]=imp_train_df.quantile(q=0.2,axis=1)
imp_train_df["q25"]=imp_train_df.quantile(q=0.25,axis=1)
imp_train_df["q3"]=imp_train_df.quantile(q=0.3,axis=1)
imp_train_df["q4"]=imp_train_df.quantile(q=0.4,axis=1)
imp_train_df["q5"]=imp_train_df.quantile(q=0.5,axis=1)
imp_train_df["q6"]=imp_train_df.quantile(q=0.6,axis=1)
imp_train_df["q7"]=imp_train_df.quantile(q=0.7,axis=1)
imp_train_df["q75"]=imp_train_df.quantile(q=0.75,axis=1)
imp_train_df["q8"]=imp_train_df.quantile(q=0.8,axis=1)
imp_train_df["q9"]=imp_train_df.quantile(q=0.9,axis=1)

In [147]:
imp_test_df["mean"]=imp_test_df.mean(axis=1)
imp_test_df["count"]=imp_test_df.count(axis=1)
imp_test_df["std"]=imp_test_df.std(axis=1)
imp_test_df["min"]=imp_test_df.min(axis=1)
imp_test_df["max"]=imp_test_df.max(axis=1)
imp_test_df["kurt"]=imp_test_df.kurt(axis=1)
imp_test_df["skew"]=imp_test_df.skew(axis=1)
imp_test_df["q1"]=imp_test_df.quantile(q=0.1,axis=1)
imp_test_df["q2"]=imp_test_df.quantile(q=0.2,axis=1)
imp_test_df["q25"]=imp_test_df.quantile(q=0.25,axis=1)
imp_test_df["q3"]=imp_test_df.quantile(q=0.3,axis=1)
imp_test_df["q4"]=imp_test_df.quantile(q=0.4,axis=1)
imp_test_df["q5"]=imp_test_df.quantile(q=0.5,axis=1)
imp_test_df["q6"]=imp_test_df.quantile(q=0.6,axis=1)
imp_test_df["q7"]=imp_test_df.quantile(q=0.7,axis=1)
imp_test_df["q75"]=imp_test_df.quantile(q=0.75,axis=1)
imp_test_df["q8"]=imp_test_df.quantile(q=0.8,axis=1)
imp_test_df["q9"]=imp_test_df.quantile(q=0.9,axis=1)

In [148]:
imp_train=imp_train_df.drop("y",axis=1)

In [149]:
imp_train.shape, imp_test_df.shape

((45000, 34), (15000, 34))

In [150]:
y_train=imp_train_df["y"]

In [151]:
imp_train_df["y"]

0         4500.0
1        22133.0
2        11490.0
3        14500.0
4        10000.0
          ...   
44995    23790.0
44996    31212.0
44997    32590.0
44998    23812.0
44999    26395.0
Name: y, Length: 45000, dtype: float64

In [152]:
col=imp_test_df.columns  
test_ = imp_test_df[col]


In [153]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

from tqdm import tqdm_notebook
import lightgbm as lgb
from catboost import Pool, CatBoostRegressor

In [154]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold,StratifiedKFold ,GroupKFold
from sklearn.metrics import mean_squared_error
from sklearn import  ensemble
def get_model(Name='lgbm') :
    if Name=='lgbm' :
      return LGBMRegressor(**{'objective' :'regression','boosting_type' : 'gbdt','metric': 'rmse' ,
                              'learning_rate' : 0.05,'num_iterations': 1500,'max_depth' :8 ,'num_leaves' : 150,
                              'max_bins': 85,'min_data_in_leaf':30,'reg_lambda' :75})

In [155]:
Model_Name = "lgbm"
folds = KFold(n_splits =8,shuffle=True,random_state=160)
oofs  = np.zeros((len(imp_train[col])))
test_predictions = np.zeros((len(imp_test_df)))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(imp_train, imp_train_df['y'])):
    X_train, y_train = imp_train.iloc[trn_idx][col], imp_train_df.iloc[trn_idx]['y']
    X_valid, y_valid = imp_train.iloc[val_idx][col], imp_train_df.iloc[val_idx]['y']
#     X_trn, y_trn = X.iloc[trn_idx], y.iloc[trn_idx]
#     X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
    
    clf = get_model(Name=Model_Name)
    clf.fit(X_train, y_train, eval_set = [(X_valid, y_valid)],
            verbose =100, early_stopping_rounds = 200)
    
    vp = clf.predict(X_valid)
    oofs[val_idx] = vp
    val_score = mean_squared_error((vp), (y_valid),squared=True)
    print(4*'-- -- -- --')
    print(f'Fold {fold_+1} Val score: {val_score}')
    print(4*'-- -- -- --')
    
    tp = clf.predict(test_)
    test_predictions += tp / folds.n_splits

  
print()
print(3*'###',10*"^",3*'###')
print(mean_squared_error(imp_train_df["y"], oofs,squared=True))
print("Model training")

[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[100]	valid_0's rmse: 1551.59
[200]	valid_0's rmse: 1334.91
[300]	valid_0's rmse: 1267.48
[400]	valid_0's rmse: 1237.97
[500]	valid_0's rmse: 1218.9
[600]	valid_0's rmse: 1203.65
[700]	valid_0's rmse: 1192.18
[800]	valid_0's rmse: 1181.7
[900]	valid_0's rmse: 1172.32
[1000]	valid_0's rmse: 1163.5
[1100]	valid_0's rmse: 1156.05
[1200]	valid_0's rmse: 1148.96
[1300]	valid_0's rmse: 1142.06
[1400]	valid_0's rmse: 1135.87
[1500]	valid_0's rmse: 1130.57
-- -- -- ---- -- -- ---- -- -- ---- -- -- --
Fold 1 Val score: 1278020.4705091985
-- -- -- ---- -- -- ---- -- -- ---- -- -- --
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[100]	valid_0's rmse: 896.005
[200]	valid_0's rmse: 652.62
[300]	valid_0's rmse: 582.779
[400]	valid_0's rmse: 555.388
[500]	valid_0's rmse: 542.663
[600]	valid_0's rmse: 535.3

In [158]:
skf = KFold(n_splits=3, shuffle=True, random_state=5168)
for fold, (trn_idx, val_idx) in enumerate(skf.split(imp_train, imp_train_df)):
    X_train, y_train = imp_train.iloc[trn_idx][cols], imp_train_df.iloc[trn_idx]['y']
    X_valid, y_valid = imp_train.iloc[val_idx][cols], imp_train_df.iloc[val_idx]['y']
    break
    
    
    
clf = CatBoostRegressor(loss_function = "RMSE", eval_metric = "MAE",random_seed=123,use_best_model=True,
                          learning_rate=0.01,  iterations=25000,verbose=100,
                           bootstrap_type= "Poisson", 
                           task_type="GPU", 
#                              l2_leaf_reg= 16.5056753964314982, depth= 3.0,
#                              fold_len_multiplier= 2.9772639036842174, 
#                              scale_pos_weight= 3.542962442406767, 
#                              fold_permutation_block_size=16.0, subsample= 0.46893530376570957
#                              fold_len_multiplier=3.2685541035861747, 
#                              scale_pos_weight= 2.6496926337120916, 
#                              fold_permutation_block_size= 6.0, 
                          )
print("Model training")
clf.fit(X_train, y_train,  eval_set=(X_valid, y_valid), early_stopping_rounds=2000,verbose=100)

Model training


Default metric period is 5 because MAE is/are not implemented for GPU


0:	learn: 7614.9648000	test: 7583.8581333	best: 7583.8581333 (0)	total: 21ms	remaining: 8m 44s
100:	learn: 3670.6434667	test: 3669.9064000	best: 3669.9064000 (100)	total: 1.82s	remaining: 7m 28s
200:	learn: 2146.8309333	test: 2150.3429333	best: 2150.3429333 (200)	total: 3.55s	remaining: 7m 17s
300:	learn: 1486.1737333	test: 1497.1505333	best: 1497.1505333 (300)	total: 5.42s	remaining: 7m 25s
400:	learn: 1195.6438667	test: 1212.4141333	best: 1212.4141333 (400)	total: 7.45s	remaining: 7m 36s
500:	learn: 1041.7186667	test: 1061.6972000	best: 1061.6972000 (500)	total: 9.24s	remaining: 7m 32s
600:	learn: 929.3049333	test: 951.7126667	best: 951.7126667 (600)	total: 11s	remaining: 7m 24s
700:	learn: 849.8289333	test: 873.4734667	best: 873.4734667 (700)	total: 12.7s	remaining: 7m 19s
800:	learn: 781.9608000	test: 806.2032667	best: 806.2032667 (800)	total: 14.4s	remaining: 7m 15s
900:	learn: 732.0127333	test: 756.2450000	best: 756.2450000 (900)	total: 16.1s	remaining: 7m 11s
1000:	learn: 694.87

In [159]:
prediction = clf.predict(test_[col])


In [160]:
models = []
models.append(CatBoostRegressor(random_state=17, max_depth = 3))
models.append(CatBoostRegressor(random_state=18, num_trees=700, max_depth=5))
models.append(CatBoostRegressor(random_state=19, num_trees=500, max_depth = 3))
models.append(CatBoostRegressor(random_state=20, num_trees=1200, max_depth=3))

In [161]:
for i in range(len(models)):
  models[i].fit(imp_train, imp_train_df['y'])

Learning rate set to 0.074711
0:	learn: 9366.3938462	total: 16.3ms	remaining: 16.3s
1:	learn: 8906.5778914	total: 26.2ms	remaining: 13.1s
2:	learn: 8491.0420236	total: 35.7ms	remaining: 11.9s
3:	learn: 8087.8876066	total: 44.9ms	remaining: 11.2s
4:	learn: 7745.2157219	total: 54.1ms	remaining: 10.8s
5:	learn: 7407.5898137	total: 62.7ms	remaining: 10.4s
6:	learn: 7122.8539946	total: 72ms	remaining: 10.2s
7:	learn: 6838.6555706	total: 80.7ms	remaining: 10s
8:	learn: 6588.6603189	total: 89.6ms	remaining: 9.86s
9:	learn: 6371.7362982	total: 98.9ms	remaining: 9.79s
10:	learn: 6142.9644501	total: 109ms	remaining: 9.79s
11:	learn: 5953.9105393	total: 118ms	remaining: 9.71s
12:	learn: 5765.8125395	total: 127ms	remaining: 9.6s
13:	learn: 5583.6829064	total: 136ms	remaining: 9.55s
14:	learn: 5420.6107980	total: 145ms	remaining: 9.51s
15:	learn: 5274.3797925	total: 153ms	remaining: 9.43s
16:	learn: 5150.7992459	total: 163ms	remaining: 9.4s
17:	learn: 5018.3271241	total: 171ms	remaining: 9.34s
18:	

In [167]:
subb=pd.DataFrame({"_id":Id,"ypred":test_predictions})

In [171]:
from IPython.display import FileLink
def create_submission(submission_file, submission_name):
    submission_file.to_csv(submission_name+".csv",index=False)
    return FileLink(submission_name+".csv")
create_submission(subb, "best_sub")

/kaggle/working/best_sub.csv